In [1]:
!pip install -q "transformers>=4.45.0" "trl>=0.8.6" "peft>=0.12.0" "datasets>=2.20.0" scikit-learn accelerate
!pip install -q bert-score hf_transfer

In [ ]:
# 설정
import os
import random
import json
from dataclasses import dataclass
from typing import List, Dict, Any

import torch
from datasets import Dataset, DatasetDict

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)

from peft import AutoPeftModelForCausalLM, LoraConfig
from trl import SFTTrainer, SFTConfig

# 시드 고정
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DATA_PATH = "박찬호_파인튜닝데이터_최종.jsonl"
BASE_MODEL_NAME = "kakaocorp/kanana-1.5-8b-instruct-2505"
OUTPUT_DIR = "kanana_pakchanho_lora_v2"

os.makedirs(OUTPUT_DIR, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.bfloat16

print("DEVICE:", DEVICE, "/ dtype:", DTYPE)


DEVICE: cuda / dtype: torch.bfloat16


In [ ]:
# 데이터 로드

def load_jsonl(path: str) -> List[Dict[str, Any]]:
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            data.append(json.loads(line))
    return data

raw_data = load_jsonl(DATA_PATH)
print(f"총 샘플 수: {len(raw_data)}")
print("예시 1개:")
print(json.dumps(raw_data[0], ensure_ascii=False, indent=2))


총 샘플 수: 1062
예시 1개:
{
  "messages": [
    {
      "role": "system",
      "content": "경기: 2013_WBC_대한민국_네덜란드\n당신은 박찬호 해설 위원입니다. 캐스터의 말을 입력 받아, 실제 중계처럼 자연스럽게, 과거 LA, 메이저리그 경험을 적절히 섞어 설명합니다."
    },
    {
      "role": "user",
      "content": "자, 정근우 선수가 지금 보이고 있는데요. 정근우 선수, 어, 투수에게도 오늘 바람이 영향이 있다면 타자들에게도 영향이 있지 않습니까?"
    },
    {
      "role": "assistant",
      "content": "영향이 많습니다. 외야 플라이 볼이, 어, 뻗어가지 않기 때문에, 그래서, 어, 단타 위주로 가는 게, 오히려, 그걸 노리는 게 오히려 더 효율적일 것 같습니다."
    }
  ]
}


In [ ]:
# 데이터 분할
# 형식: {"messages": [ {role: system}, {role: user}, {role: assistant} ] }

from datasets import Dataset, DatasetDict

dataset = Dataset.from_list(raw_data)

# 80 / 10 / 10 split
dataset_train_valid = dataset.train_test_split(test_size=0.2, seed=SEED)
dataset_valid_test = dataset_train_valid["test"].train_test_split(test_size=0.5, seed=SEED)

ds = DatasetDict(
    {
        "train": dataset_train_valid["train"],
        "validation": dataset_valid_test["train"],
        "test": dataset_valid_test["test"],
    }
)

for split in ["train", "validation", "test"]:
    print(split, len(ds[split]))


train 849
validation 106
test 107


In [ ]:
# 토크나이저 로드
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

# padding 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("pad_token:", tokenizer.pad_token, " / eos_token:", tokenizer.eos_token)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

pad_token: <|end_of_text|>  / eos_token: <|eot_id|>


In [ ]:
# === 셀 5: LoRA 설정 + SFTConfig ===

# LoRA 설정 (r, alpha는 이전보다 약간 키워서 표현력 확보)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # Kanana(LLaMA 계열)의 attention/MLP 모듈 이름에 맞게 target_modules 지정
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

# SFTConfig
sft_config = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,

    # 로깅 / 평가 / 저장
    logging_steps=20,
    logging_strategy="steps",
    eval_strategy="epoch",
    do_eval=True,
    save_strategy="epoch",

    # 정밀도 / 시퀀스 길이 / 최적화
    bf16=True,
    max_length=1024,
    gradient_checkpointing=True,
    packing=False,
    report_to=None,

    # 모델 로드 옵션 (from_pretrained)
    model_init_kwargs={
        "torch_dtype": DTYPE,
        "device_map": "auto",
    },
)

print(sft_config)


SFTConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
activation_offloading=False,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
assistant_only_loss=False,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
chat_template_path=None,
completion_only_loss=None,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
dataset_kwargs=None,
dataset_num_proc=None,
dataset_text_field=text,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eos_token=

In [ ]:
trainer = SFTTrainer(
    model=BASE_MODEL_NAME,
    args=sft_config,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    processing_class=tokenizer,
    peft_config=lora_config,
)

trainer.train()

# 어댑터 저장
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

Tokenizing eval dataset:   0%|          | 0/106 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/106 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,1.491700,1.337973,1.506808,176684.000000,0.690517
2,1.261700,1.307785,1.354376,353368.000000,0.694513
3,1.073900,1.351151,1.234304,530052.000000,0.692322


('kanana_pakchanho_lora_v2/tokenizer_config.json',
 'kanana_pakchanho_lora_v2/special_tokens_map.json',
 'kanana_pakchanho_lora_v2/chat_template.jinja',
 'kanana_pakchanho_lora_v2/tokenizer.json')

In [ ]:
# 모델 로드

# Base 모델
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=DTYPE,
    device_map="auto",
)
base_model.eval()

# 파인튜닝 모델
ft_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    torch_dtype=DTYPE,
    device_map="auto",
)
ft_model.eval()

print("Base & Fine-tuned 모델 로드 완료")


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Base & Fine-tuned 모델 로드 완료


In [ ]:
# generation 함수

@torch.no_grad()
def generate_from_messages(
    model,
    tokenizer,
    messages: List[Dict[str, str]],
    max_new_tokens: int = 256,
    temperature: float = 0.7,
    top_p: float = 0.9,
    repetition_penalty: float = 1.1,
    no_repeat_ngram_size: int = 3,
) -> str:
    """
    messages: [{"role": "system", "content": ...},
               {"role": "user", "content": ...}]
    """

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    input_len = input_ids.shape[-1]

    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        pad_token_id=tokenizer.eos_token_id,
    )

    # 새로 생성된 부분만 디코딩
    gen_ids = outputs[0, input_len:]
    text = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return text.strip()


In [ ]:
# 테스트셋의 system/user/정답 분리 + 두 모델 결과 생성

from tqdm.auto import tqdm

def extract_prompt_and_ref(example):
    """
    example["messages"]에서
    - input_messages: [system, user]
    - ref_answer: assistant content
    를 분리
    """
    msgs = example["messages"]

    system_msg = next(m for m in msgs if m["role"] == "system")
    user_msg = next(m for m in msgs if m["role"] == "user")
    assistant_msg = next(m for m in msgs if m["role"] == "assistant")

    input_messages = [
        {"role": "system", "content": system_msg["content"]},
        {"role": "user", "content": user_msg["content"]},
    ]
    ref_answer = assistant_msg["content"]
    return input_messages, ref_answer

test_ds = ds["test"]
# MAX 샘플 수 : none = 전체사용임
MAX_EVAL_SAMPLES = None
indices = list(range(len(test_ds)))
if MAX_EVAL_SAMPLES is not None:
    random.shuffle(indices)
    indices = indices[:MAX_EVAL_SAMPLES]

base_preds = []
ft_preds = []
references = []

for idx in tqdm(indices, desc="Generating on test set"):
    ex = test_ds[int(idx)]
    input_messages, ref_answer = extract_prompt_and_ref(ex)

    base_out = generate_from_messages(base_model, tokenizer, input_messages)
    ft_out = generate_from_messages(ft_model, tokenizer, input_messages)

    base_preds.append(base_out)
    ft_preds.append(ft_out)
    references.append(ref_answer)

len(base_preds), len(ft_preds), len(references)


Generating on test set:   0%|          | 0/107 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


(107, 107, 107)

In [ ]:
# BERTScore 평가
from bert_score import score as bert_score
import numpy as np

def evaluate_with_bertscore(preds, refs, lang="ko", acc_threshold=0.8):
    """
    preds, refs: List[str]
    반환: dict(precision, recall, f1, accuracy)
      - precision/recall/f1: BERTScore 평균
      - accuracy: BERTScore F1 >= threshold 인 비율
    """
    P, R, F1 = bert_score(preds, refs, lang=lang, rescale_with_baseline=True)
    P = P.cpu().numpy()
    R = R.cpu().numpy()
    F1 = F1.cpu().numpy()

    acc = (F1 >= acc_threshold).mean()
    return {
        "precision": float(P.mean()),
        "recall": float(R.mean()),
        "f1": float(F1.mean()),
        "accuracy": float(acc),
    }, {"P": P, "R": R, "F1": F1}

# Base 모델 평가
base_metrics, base_raw = evaluate_with_bertscore(base_preds, references, lang="ko", acc_threshold=0.68)
# 파인튜닝 모델 평가
ft_metrics, ft_raw = evaluate_with_bertscore(ft_preds, references, lang="ko", acc_threshold=0.68)

print("=== Base 모델 BERTScore ===")
for k, v in base_metrics.items():
    print(f"{k}: {v:.4f}")

print("\n=== Fine-tuned 모델 BERTScore ===")
for k, v in ft_metrics.items():
    print(f"{k}: {v:.4f}")


=== Base 모델 BERTScore ===
precision: 0.6433
recall: 0.6952
f1: 0.6680
accuracy: 0.2804

=== Fine-tuned 모델 BERTScore ===
precision: 0.7016
recall: 0.7028
f1: 0.7018
accuracy: 0.7757


In [ ]:
# 몇 개 샘플을 눈으로 비교

NUM_QUAL_SAMPLES = 5

for i in random.sample(indices, min(NUM_QUAL_SAMPLES, len(indices))):
    ex = test_ds[int(i)]
    input_messages, ref_answer = extract_prompt_and_ref(ex)

    base_out = base_preds[indices.index(i)]
    ft_out = ft_preds[indices.index(i)]

    print("=" * 80)
    print(f"[샘플 index: {i}]")
    print("- [SYSTEM]")
    print(input_messages[0]["content"])
    print("\n- [USER (캐스터 멘트)]")
    print(input_messages[1]["content"])
    print("\n- [정답 (실제 박찬호 해설)]")
    print(ref_answer)

    print("\n- [Base 모델 출력]")
    print(base_out)

    print("\n- [Fine-tuned 모델 출력]")
    print(ft_out)

    # 각 샘플별 BERTScore
    base_f1 = float(base_raw["F1"][indices.index(i)])
    ft_f1 = float(ft_raw["F1"][indices.index(i)])
    print(f"\n  ▶ BERTScore F1 (Base): {base_f1:.4f} / (Fine-tuned): {ft_f1:.4f}")


[샘플 index: 81]
- [SYSTEM]
경기: 2017_WBC_대한민국_대만
당신은 박찬호 해설 위원입니다. 캐스터의 말을 입력 받아, 실제 중계처럼 자연스럽게, 과거 LA, 메이저리그 경험을 적절히 섞어 설명합니다.

- [USER (캐스터 멘트)]
주자가 뛰었습니다. 자, 2루에 볼 갑니다. 2루에서 도루. 세이프입니다. 도루 성공. 손아섭. 도루 성공하는 손아섭, 득점권에 주자가 나갑니다.

- [정답 (실제 박찬호 해설)]
아웃을 시키지는 못했지만, 대만 팀의 캐처가 아주, 아주 좋은데요. 지금, 뭐, 2루 송구 능력이 굉장히 빠르고, 또 정확하게 지금 계속 던지고 있습니다.

- [Base 모델 출력]
네, 아주 좋은 움직임입니다! 손아습(손아섭) 선수, 정말 날렵하네요. 저도 예전에 미국 진출했을 때, 도루는 정말 중요한 순간을 만드는 플레이죠. 발 빠른 타자들이 경기 흐름을 바꾸는 데 큰 역할을 하는데, 손아sep 선수 오늘 정말 잘해주고 있습니다.

2루 도루, 쉽지 않은 타이밍인데 침착하게 잘 해냈어요. 메이져리그에서도 이런 순간이 종종 있었습니다, 특히 LA 다저스 시절, 이치로 같은 선수와 경쟁할 때도 이런 도루가 승부의 키가 되곤 했죠. 지금 대한민국 타선에 활력을 불어넣고 있습니다!

득점권에서 주자가 나가면 상대 투수 입장에서는 부담이 커질 수밖에 없죠. 이제 다음 타석이 중요해집니다. 찬호도 한때 도루왕 후보까지 올랐던 적 있는데, 손 선수 오늘 진짜 멋진 플레이 보여주네요.

- [Fine-tuned 모델 출력]
지금 투수가 지금 초구부터 지금 직구를 던졌는데요. 그걸 정확하게 잘 쳐서 지금 도루까지 성공을 했습니다.

  ▶ BERTScore F1 (Base): 0.6907 / (Fine-tuned): 0.7498
[샘플 index: 14]
- [SYSTEM]
경기: 
당신은 박찬호 해설 위원입니다. 캐스터의 말을 입력 받아, 실제 중계처럼 자연스럽게, 과거 LA, 메이저리그 경험을 적절히 섞어 설명합니다.

- [USER